In [1]:
#Homepages:
    # https://nikolanews.com/develop-a-blockchain-application-from-scratch-in-python/ 
    # beschreibe das was und nicht das wie!

In [2]:
import json # einfachere Handhabung als xml, einfacher schr.

In [3]:
from hashlib import sha256 # fixed 256 bit string - hexadezimal - 64 random character string

In [4]:
from time import time 

In [5]:
class Block:
    def __init__(self, uid, timestamp, transactions, previous_hash):
        self.uid = uid # um Block zu identifizieren
        self.timestamp = timestamp
        self.transations = transactions # sind die Daten welche in Blöcken gespeichert werden. i.d.F Transaktionsdaten
        self.previous_hash = previous_hash # Wenn sich Hashwert des vorherigen Blocks ändern würde, ändert sich jene des aktuellen auch
        self.nonce = None # unique variable to be found
        
    def compute_hash(self):
        data = json.dumps(self.__dict__) # Konvertiert json Objekt in python string
        encoded = data.encode('UTF-8') # Kodierung für Unicode zeichen
        return sha256(encoded).hexdigest() # Ausgabe, stets die gleiche, egal wie häufig durchlaufen

# Frage: Hier würde Root tree verwendet werden, wenn individuelle Transaktionen gehashed werden????

In [6]:
class BlockChain: # typ mit großem C, variable etc.klein
    difficulty = 2 # je höher, desto schwerer --> warum steht sie nicht direkt im Block? kompliziert wenn man 2 haben will
    
    def __init__(self):
        self.chain = [BlockChain.create_genesis_block()] # um Blockchain zu erstellen und Genesis Block ergänzen 
    
    @staticmethod # --> bedeutet es, das habe ich immer?
    def create_genesis_block():
        """Erster Block einer Blockchain, bezieht sich nicht auf vorherigen Block. Index = 0""" # auf englisch
        return Block(0, time(), [], 'G E N E S I S')
    
    @property  # nicht sooo wichtig
    def last_block(self):
        return self.chain[-1] 

# Frage: ht ganz klar? -1 um aktuellen Block auszugeben? ist letztes element

    @staticmethod
    def proof_of_work(block): # verhindert, dass hashes verändert werden können
        block.nonce = 0
        computed_hash = block.compute_hash()
        while not BlockChain.is_valid_hash(computed_hash): # checkt ob Hash mit 00 beginnt (wegen difficulty) ist..solange nicht Blockchain valide ist, suche weiter 
            block.nonce += 1 # so wird nonce immer weiter gesucht und plus 1 gezählt
            computed_hash = block.compute_hash()
        return computed_hash # gefundene Hash Ausgabe
    
    @staticmethod
    def is_valid_hash(block_hash): 
        return block_hash.startswith('0' * BlockChain.difficulty) # Kontrolle ob hash valide ist von Block und Difficulty erfüllt

    

In [11]:
def main():
#create chain
    blockchain = BlockChain()
    print(f'blockchain = {blockchain.__dict__}') 
    
#create block
    uid = 1 
    timestamp = time()
    transactions = ['{"name": "Transaction 1"}']
    previous_hash = blockchain.last_block().compute_hash() # hash des letzten blocks wird neu berechnet aber nicht p.o.w
    block = Block(uid, timestamp, transactions, previous_hash)
    print(f'block = {block.__dict__}')
    
#create proof of work
    computed_hash = blockchain.proof_of_work(block) 
    print('computed_hash = ' + computed_hash) 
    
#append Block
    blockchain.chain.append(block)
    print(f'blockchain = {blockchain.__dict__}')
    
# verify blockchain
    last_hash = None
    for b in blockchain.chain:
        block_hash = b.compute_hash()
        if b.uid > 0:
            if not BlockChain.is_valid_hash(block_hash):
                raise Exception('Invalid hash')
            if last_hash !=b.previous_hash:
                raise Exception('invalid chain')
        elif b.uid == 0:
            pass # genesis block, no need to do smthg.
        else:
            raise Exception('Invalid uid')
        last_hash = block_hash
    
    print('blockchain is valid')
    
if __name__ == '__main__':
    main()


    

blockchain = {'chain': [<__main__.Block object at 0x7f3244680240>]}


TypeError: 'Block' object is not callable